# Multi Agent Team Approach - With Termination Condition

In [4]:
import asyncio
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import (AssistantAgent)
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [3]:
# Specify Agents
plot_agent = AssistantAgent(
    name= 'plot_writer',
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on being a murder mystery detective chef."
)

character_agent = AssistantAgent(
    name='character_writer',
    model_client=model_client,
    system_message="You develop characters. Describe the chefs and the murderer in detail, including their motivations and background."
)

ending_agent = AssistantAgent(
    name='ending_writer',
    model_client=model_client,
    system_message="You write engaing endings. conclude the story with a twist."
)

## Termination Condition: MaxMessageTermination
Stops after a specified number of messages have been produced including both agent and task messages

In [8]:
from autogen_agentchat.conditions import MaxMessageTermination

# Create termination condition
# NOTE: necessary, as max_turns only available in RoundRobinGroupChat and not other GroupChats
termination_condition = MaxMessageTermination(2)

In [9]:
# Establish team
team = RoundRobinGroupChat(
    participants=[plot_agent, character_agent, ending_agent],
    termination_condition=termination_condition
    #max_turns=3
)


In [10]:
from autogen_agentchat.base import TaskResult

# When running a script, use a async main function and call it from `asyncio.run(...)`
await team.reset()   # Reset the team for a new task

async for message in team.run_stream(task="Write a short poam about the fall season."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='70bb576a-4a17-42a5-bff3-11d1d39de296' source='user' models_usage=None metadata={} created_at=datetime.datetime(2026, 1, 22, 13, 40, 44, 490215, tzinfo=datetime.timezone.utc) content='Write a short poam about the fall season.' type='TextMessage'
id='e4985283-99d4-4de1-b79c-b72663790d49' source='plot_writer' models_usage=RequestUsage(prompt_tokens=38, completion_tokens=182) metadata={} created_at=datetime.datetime(2026, 1, 22, 13, 40, 51, 288019, tzinfo=datetime.timezone.utc) content="In whispers of gold and crisp, the leaves descend,  \nA tapestry woven where summer dreams end.  \nMisty breath tangles with cinnamon's charm,  \n'Tween the scent of ripe apples, fall wraps warm.  \n\nThrough woods rich with russet and auburn gleam,  \nDaylight meanders in a pumpkin spice dream.  \nPainted skies blush with twilight's soft flare,  \nWhile shadows dance in the cool autumn air.  \n\nChilled gusts rattle through branches once green,  \nScattering secrets where silence convene.  \nIn the hea

## Multiple Termination Conditions
Can combine termination conditions

In [13]:
from autogen_agentchat.conditions import TextMentionTermination

termination_condition = MaxMessageTermination(10) | TextMentionTermination('APPROVE')

In [14]:
# Createa new agent
review_agent = AssistantAgent(
    name = 'review_writer',
    model_client=model_client,
    system_message="You have to say 'APPROVE' once the story is complete or you will have to provide feedback"
)

# Establish team
team = RoundRobinGroupChat(
    participants=[plot_agent, character_agent, ending_agent, review_agent],
    termination_condition=termination_condition
    #max_turns=3
)

# Run
await team.reset()   # Reset the team for a new task
async for message in team.run_stream(task="Write a short poam about the fall season."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='7cccd5f8-632f-4286-8c1b-fb90aeacaacd' source='user' models_usage=None metadata={} created_at=datetime.datetime(2026, 1, 22, 16, 19, 20, 12745, tzinfo=datetime.timezone.utc) content='Write a short poam about the fall season.' type='TextMessage'
id='e9292e14-656a-4c5c-8f22-7a41968b5e0b' source='plot_writer' models_usage=RequestUsage(prompt_tokens=38, completion_tokens=115) metadata={} created_at=datetime.datetime(2026, 1, 22, 16, 19, 23, 719364, tzinfo=datetime.timezone.utc) content='Autumn whispers in hues of gold,  \nLeaves rustle secrets, tales retold.  \nPumpkin spice and chimneys smoke,  \nIn chilly gusts the old trees cloak.  \n\nCrisp air bites with a gentle nudge,  \nOrchards boast of cider’s grudge.  \nWarm woolen knit and fading light,  \nStars sprinkle tales in the brisk night.  \n\nA tapestry of amber leaves,  \nWeaves a story the heart believes.  \nIn nature’s dance, a symphony,  \nFall sings its sweet soliloquy.  ' type='TextMessage'
id='e68dc82a-e194-4a32-98c6-0dd318ee